In [20]:
import mlflow
import mlflow.sklearn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import joblib
import sagemaker
from sagemaker import ModelPackage
from sagemaker.image_uris import retrieve
bucket_name = 'sagemaker-studio-42jgegdgih'

In [3]:
# Load Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize and train a logistic regression model with increased max_iter
model = LogisticRegression(max_iter=1000)  # Increase the number of iterations
model.fit(X_train_scaled, y_train)  # Use scaled data

# Make predictions and evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
joblib.dump(model, 'model.joblib')

['model.joblib']

In [21]:
import boto3

client = boto3.client('iam')

response = client.list_roles()

#for role in response['Roles']:
   #print("Role name: {}, ARN: {}".format(role['RoleName'], role['Arn']))
    
role = 'arn:aws:iam::870665956147:role/service-role/AmazonSageMaker-ExecutionRole-20231023T114118'

In [22]:
from sagemaker import Model, Session
from sagemaker.image_uris import retrieve
import boto3

# Initialize variables
model_artifact = f's3://{bucket_name}/model/model.tar.gz'

# Initialize the SageMaker Session
sagemaker_session = Session()

model_package_group_name = 'MyModelPackageGroup1'

sagemaker_client = boto3.client('sagemaker')

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [14]:
model_data

's3://sagemaker-studio-42jgegdgih/model/model.tar.gz'

In [23]:
# Get the Docker image URI for the Scikit-Learn framework
image_uri = retrieve('sklearn', region='us-east-2', version='0.23-1')

In [24]:
import boto3
import sagemaker
from sagemaker import get_execution_role

sagemaker_client = boto3.client('sagemaker')
sagemaker_session = sagemaker.Session()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [25]:
def get_latest_model_package_arn(model_package_group_name):
    response = sagemaker_client.list_model_packages(
        ModelPackageGroupName=model_package_group_name,
        SortBy='CreationTime',
        SortOrder='Descending',
        MaxResults=1
    )
    
    if response['ModelPackageSummaryList']:
        return response['ModelPackageSummaryList'][0]['ModelPackageArn']
    else:
        return None

model_package_arn = get_latest_model_package_arn(model_package_group_name)
model_package_arn


'arn:aws:sagemaker:us-east-2:870665956147:model-package/MyModelPackageGroup1/2'

In [26]:
import boto3

def approve_model_version(model_package_arn):
    sagemaker_client = boto3.client('sagemaker')

    # Update the model package to set the approval status to "Approved"
    sagemaker_client.update_model_package(
        ModelPackageArn=model_package_arn,
        ModelApprovalStatus='Approved'  # Set to 'Approved' status
    )

# Use the function
approve_model_version(model_package_arn)

In [27]:
import os
import boto3
from urllib.parse import urlparse

def download_from_s3(s3_uri, local_path):
    # Parse the S3 URI
    parsed_url = urlparse(s3_uri)
    bucket_name = parsed_url.netloc
    s3_key = parsed_url.path.lstrip('/')
    
    # Initialize S3 client and download the file
    s3_client = boto3.client('s3')
    s3_client.download_file(bucket_name, s3_key, local_path)
    
# Define local path for download
local_model_path = 'model.tar.gz'

# Download the model
download_from_s3(model_artifact_s3_uri, local_model_path)

print(f"Model downloaded to: {local_model_path}")


Model downloaded to: model.tar.gz


In [28]:
import tarfile

with tarfile.open(local_model_path, 'r:gz') as tar:
    tar.extractall(path='.')

print("Model extracted.")


Model extracted.


In [29]:
import joblib

model_filename = 'model.joblib'  # This is the usual naming convention, but it might differ.
loaded_model = joblib.load(model_filename)

print("Model loaded.")


Model loaded.


In [31]:
loaded_model

LogisticRegression()

In [30]:
# Sample input for prediction. Modify it as per your use case.
sample_input = [[5.1, 3.5, 1.4, 0.2]]

predictions = loaded_model.predict(sample_input)

print("Predictions:", predictions)


Predictions: [0]
